# 10Q Analysis
In this demo, we explore answering complex queries by decomposing them into simpler sub-queries.

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [13]:
from llama_index import SimpleDirectoryReader, LLMPredictor, ServiceContext, GPTVectorStoreIndex
from llama_index.response.pprint_utils import pprint_response
from langchain import OpenAI

from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

## Configure LLM service

In [4]:
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=-1, streaming=True))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

## Load data

In [6]:
march_2022 = SimpleDirectoryReader(input_files=["../data/10q/uber_10q_march_2022.pdf"]).load_data()
june_2022 = SimpleDirectoryReader(input_files=["../data/10q/uber_10q_june_2022.pdf"]).load_data()
sept_2022 = SimpleDirectoryReader(input_files=["../data/10q/uber_10q_sept_2022.pdf"]).load_data()

# Build indices

In [8]:
march_index = GPTVectorStoreIndex.from_documents(march_2022)
june_index = GPTVectorStoreIndex.from_documents(june_2022)
sept_index = GPTVectorStoreIndex.from_documents(sept_2022)

## Build query engines

In [9]:
march_engine = march_index.as_query_engine(similarity_top_k=3)
june_engine = june_index.as_query_engine(similarity_top_k=3)
sept_engine = sept_index.as_query_engine(similarity_top_k=3)

In [27]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=sept_engine, 
        metadata=ToolMetadata(name='sept_22', description='Provides information about Uber quarterly financials ending September 2022')
    ),
    QueryEngineTool(
        query_engine=june_engine, 
        metadata=ToolMetadata(name='june_22', description='Provides information about Uber quarterly financials ending June 2022')
    ),
    QueryEngineTool(
        query_engine=march_engine, 
        metadata=ToolMetadata(name='march_22', description='Provides information about Uber quarterly financials ending March 2022')
    ),
]

In [28]:
s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

## Run queries

In [35]:
response = s_engine.query('Analyze Uber revenue growth over the latest two quarter filings')

Generated 2 sub questions.
[sept_22] Q: What is the revenue growth of Uber for the quarter ending September 2022
[sept_22] A: compared to the same period in 2021?

The revenue growth of Uber for the quarter ending September 2022 compared to the same period in 2021 is 72%.
[june_22] Q: What is the revenue growth of Uber for the quarter ending June 2022
[june_22] A: compared to the same period in 2021?

The revenue growth of Uber for the quarter ending June 2022 compared to the same period in 2021 is 105%.


In [36]:
print(response)


Uber's revenue growth over the latest two quarter filings has been strong, with a 72% increase for the quarter ending September 2022 compared to the same period in 2021, and a 105% increase for the quarter ending June 2022 compared to the same period in 2021.


In [37]:
response = s_engine.query('Analyze change in macro environment over the 3 quarters')

Generated 3 sub questions.
[sept_22] Q: What is the macro environment in September 2022
[sept_22] A: 
The macro environment in September 2022 is one of recovery from the impacts of the COVID-19 pandemic, with increases in Mobility Trip volumes, a $1.3 billion increase in Freight Gross Bookings resulting from the acquisition of Transplace, a $1.1 billion increase in Mobility revenue due to business model changes in the UK, and a $164 million increase in Delivery revenue due to an increase in certain Courier payments and incentives. Additionally, there was a $2.2 billion net increase in Mobility revenue due to business model changes in the UK and an accrual made for the resolution of historical claims in the UK relating to the classification of drivers, and a $751 million increase in Delivery revenue due to an increase in certain Courier payments and incentives.
[june_22] Q: What is the macro environment in June 2022
[june_22] A: 
In June 2022, the macro environment is characterized by t

In [38]:
print(response)


The macro environment has seen a significant change over the three quarters from March 2022 to September 2022. In March 2022, the macro environment was uncertain due to the effects of the COVID-19 pandemic and the actions taken to mitigate it. By June 2022, the macro environment was characterized by the continued impact of COVID-19 restrictions on global demand, the adoption of new accounting standards, and the potential for shifts in consumer travel patterns due to health concerns. By September 2022, the macro environment had shifted to one of recovery from the impacts of the COVID-19 pandemic, with increases in Mobility Trip volumes, a $1.3 billion increase in Freight Gross Bookings resulting from the acquisition of Transplace, a $1.1 billion increase in Mobility revenue due to business model changes in the UK, and a $164 million increase in Delivery revenue due to an increase in certain Courier payments and incentives. Additionally, there was a $2.2 billion net increase in Mobility

In [41]:
response = s_engine.query('How much cash did Uber have in sept 2022')

Generated 1 sub questions.
[sept_22] Q: How much cash did Uber have in September 2022
[sept_22] A: 
Uber had $4,865 million in cash in September 2022.


In [42]:
print(response)


Uber had $4,865 million in cash in September 2022.
